# Step 1: Activate Virtual Environment & Run the MCP Server
```Run Command (From project-root directory):  uv run .\mcp_server\main.py```

# Step 2: Connect Server to MCP Client & Grab Tools

In [ ]:
import json
from mcp_client import McpClientPool, SEPARATOR # The separator is used for avoiding duplicate tool names. 

# NOTE: Since each server requires it's own MCP client, 
# we can manage multiple clients with an MCP client pool
# that holds all the connections. This all|ws us to 
# safely open and close all clients.
mcp_pool = McpClientPool()

# Setting up the server information 
# NOTE: Local Servers!
#   - Local servers will always have the /mcp
#   - By default, all local servers will be HTTP not HTTPS
server_id = "demo"
server_url = "http://127.0.0.1:4000/mcp" 

# Connecting to the server and saving the tools
await mcp_pool.add_client(name=server_id, base_url=server_url)

# NOTE: All the tools available also exist in the mcp_pool's instance field all_tools (mcp.all_tools).
# However, for demo purposes we will explicitly get the tools
tools = await mcp_pool.list_tools(server_id)

In [ ]:
for i, t in enumerate(tools):
    print(f"Tool {i+1}: ")
    print(json.dumps(t, indent=4), end="\n\n")

# Step 3: Ask a question

In [ ]:
query = "What is the MCP, Model Context Protocol?"

# Step 4: Pass Query and Tools to LLM & Select Best Tool

In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

openai_client = AzureOpenAI(
    api_key=os.getenv('AZURE_API_KEY'),
    api_version=os.getenv('VERSION'),
    azure_endpoint=os.getenv('ENDPOINT'),
)

response = openai_client.chat.completions.create(
    model=os.getenv('MODEL'),  
    messages=[
        # Since GPT's internal datasource is huge, sometimes the model will directly answer the query. 
        # For demo purposes we force it to select a tool via the prompt.
        {"role": "user", "content": f"You must select a tool from the provided tools given a query {query}"}
    ],
    functions=tools,
    function_call="auto"  # "auto" lets the model decide to call the tool
)

selected_tool = response.choices[0].message.function_call
print(f"Selected Tool: {selected_tool.name}")
print(f"Arguments: {selected_tool.arguments}")

# Step 5: Call The Tool!

In [ ]:
from IPython.display import display, Markdown
# Splitting the selected tool to obtain the server and the tool's name
server_name, tool_name = selected_tool.name.split(SEPARATOR)
# Loading the arguments as a JSON (str -> JSON)
loaded_args = json.loads(selected_tool.arguments)

# Calling the tool on our MCP Server
# NOTE: method is NOT the name of the tool, 
# it's the name of the route on the server
tool_response = await mcp_pool.call(
    name=server_name,
    method="tools/call",
    params={
        "name": tool_name,
        "arguments": loaded_args
    }
)

answer = tool_response[0]['result']['content'][0]['text']
display(Markdown(answer))

# Step 6: Close the Pool When Finished

In [ ]:
await mcp_pool.close_all()